# Processing VCF file using Hail

In [2]:
%scala
displayHTML("https://us.v-cdn.net/5019796/uploads/FileUpload/8d/a8143b3b21ee2f98fc83b8528f469b.png")

https://us.v-cdn.net/5019796/uploads/FileUpload/8d/a8143b3b21ee2f98fc83b8528f469b.png

In [3]:
%scala
displayHTML("https://en.wikipedia.org/wiki/Variant_Call_Format")

https://en.wikipedia.org/wiki/Variant_Call_Format

## This Notebook is based on the tutorial [Analyzing 1000 Genomes with Spark and Hail](https://docs.databricks.com/spark/latest/training/1000-genomes.html)

## Cluster setup

On the Databricks interface, click the `Clusters` icon on the left sidebar and then `+Create Cluster`. In the Databricks cluster creation dialog, click `Show advanced settings` at bottom and then on the `Spark` tab, and paste the text below into the `Spark config` box.

```
spark.hadoop.io.compression.codecs org.apache.hadoop.io.compress.DefaultCodec,is.hail.io.compress.BGzipCodec,org.apache.hadoop.io.compress.GzipCodec
spark.sql.files.openCostInBytes 1099511627776
spark.sql.files.maxPartitionBytes 1099511627776
spark.hadoop.mapreduce.input.fileinputformat.split.minsize 1099511627776
spark.hadoop.parquet.block.size 1099511627776```
Start the cluster and attach this notebook to it by clicking on your cluster name in menu `Detached` at the top left of this workbook. Now you're ready to Hail!

In [6]:
from hail import *
hc = HailContext(sc)

In [7]:
import pandas as pd
import numpy as np
from math import log, isnan
import seaborn

In [8]:
display(dbutils.fs.ls("/databricks-datasets/hail/data-001"))

path,name,size
dbfs:/databricks-datasets/hail/data-001/1kg_annotations.txt,1kg_annotations.txt,22784
dbfs:/databricks-datasets/hail/data-001/1kg_sample.vcf.bgz,1kg_sample.vcf.bgz,39767725
dbfs:/databricks-datasets/hail/data-001/purcell5k.interval_list,purcell5k.interval_list,192078


## Download the data

In [10]:
vcf_path = '/databricks-datasets/hail/data-001/1kg_sample.vcf.bgz'
annotation_path = '/databricks-datasets/hail/data-001/1kg_annotations.txt'
purcell_5k_path = '/databricks-datasets/hail/data-001/purcell5k.interval_list'

In [11]:
display(dbutils.fs.ls("/databricks-datasets/hail/data-001"))

path,name,size
dbfs:/databricks-datasets/hail/data-001/1kg_annotations.txt,1kg_annotations.txt,22784
dbfs:/databricks-datasets/hail/data-001/1kg_sample.vcf.bgz,1kg_sample.vcf.bgz,39767725
dbfs:/databricks-datasets/hail/data-001/purcell5k.interval_list,purcell5k.interval_list,192078


In [12]:
vds = hc.import_vcf(vcf_path)

This method produced a [VariantDataset](https://hail.is/hail/hail.VariantDataset.html), Hail's primary representation of genomic data. Following the link below to Hail's python API documentation will let you see the myriad methods it offers.

In [14]:
%scala 
displayHTML("https://hail.is/docs/0.1/overview.html#variant-dataset-vds")

https://hail.is/docs/0.1/overview.html#variant-dataset-vds

In [15]:
vds = vds.annotate_samples_table(annotation_path,
                                 root='sa.myAnnot',
                                 sample_expr='Sample',
                                 config=TextTableConfig(impute=True))

Use VaraintDataset function count() to get a sense of number of samples, genotypes and variants. However, any aggregate function are usually pretty slow.

In [17]:
vds.count()

Out[ 98 ]: {u'nSamples': 710, u'nVariants': 10961L, u'nGenotypes': 7782310L}

In [18]:
vds.num_samples

Out[ 99 ]: 710

In [19]:
vds.count_variants()

Out[ 54 ]: 10961L

In [20]:
dir(vds)

Out[ 100 ]: 
['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_global_schema',
 '_globals',
 '_jvds',
 '_num_samples',
 '_sa_schema',
 '_sample_annotations',
 '_sample_ids',
 '_typecheck',
 '_va_schema',
 'aggregate_by_key',
 'aggregate_intervals',
 'annotate_alleles_expr',
 'annotate_global_list',
 'annotate_global_py',
 'annotate_global_table',
 'annotate_samples_expr',
 'annotate_samples_fam',
 'annotate_samples_list',
 'annotate_samples_table',
 'annotate_samples_vds',
 'annotate_variants_bed',
 'annotate_variants_expr',
 'annotate_variants_intervals',
 'annotate_variants_keytable',
 'annotate_variants_loci',
 'annotate_variants_table',
 'annotate_variants_vds',
 'cache',
 'concordance',
 'count',
 'count_variants',
 'deduplicate',
 'downsample_variants',
 'export_gen',
 'export_genotypes',
 'export_plink',
 'export_samples',
 'export_variants',
 'export_variants_cass',
 'export_variants_solr',
 'export_vcf',
 'file_version',
 'filter_alleles',
 'filter_genotypes',
 'filter_multi',
 'filter_samples_all',
 'filter_samples_expr',
 'filter_samples_list',
 'filter_variants_all',
 'filter_variants_expr',
 'filter_variants_intervals',
 'filter_variants_list',
 'global_schema',
 'globals',
 'grm',
 'hardcalls',
 'hc',
 'ibd',
 'impute_sex',
 'is_dosage',
 'join',
 'linreg',
 'lmmreg',
 'logreg',
 'make_keytable',
 'mendel_errors',
 'min_rep',
 'num_partitions',
 'num_samples',
 'pca',
 'persist',
 'query_samples',
 'query_samples_typed',
 'query_variants',
 'query_variants_typed',
 'rename_samples',
 'repartition',
 'same',
 'sample_annotations',
 'sample_ids',
 'sample_qc',
 'sample_schema',
 'samples_keytable',
 'sparkinfo',
 'split_multi',
 'tdt',
 'variant_qc',
 'variant_schema',
 'variants_keytable',
 'vep',
 'was_split',
 'write']

### Initial exploration of dataset

If the Boolean parameter `genotypes` is set to `True`, the overall call rate across all genotypes is computed as well:

In [23]:
vds.count(genotypes=True)

Out[ 56 ]: {u'nGenotypes': 7782310L, u'nVariants': 10961L, u'nSamples': 710, u'nCalled': 7689777L, u'callRate': 98.81098285727502}

So the overall call rate before any QC filtering is about 99%.

Let's print variant and sample schemas

In [25]:
print(vds.variant_schema)

Struct {
 rsid: String,
 qual: Double,
 filters: Set[String],
 pass: Boolean,
 info: Struct {
 AC: Array[Int],
 AF: Array[Double],
 AN: Int,
 BaseQRankSum: Double,
 ClippingRankSum: Double,
 DP: Int,
 DS: Boolean,
 FS: Double,
 HaplotypeScore: Double,
 InbreedingCoeff: Double,
 MLEAC: Array[Int],
 MLEAF: Array[Double],
 MQ: Double,
 MQ0: Int,
 MQRankSum: Double,
 QD: Double,
 ReadPosRankSum: Double,
 set: String
 }
}

In [26]:
print(vds.sample_schema)

Struct {
 myAnnot: Struct {
 Sample: String,
 Population: String,
 SuperPopulation: String,
 isFemale: Boolean,
 PurpleHair: Boolean,
 CaffeineConsumption: Double
 }
}

Now it's easy to count samples by population using the [counter](https://hail.is/expr_lang.html#counter) aggregator:

In [28]:
counter = vds.query_samples('samples.map(s => sa.myAnnot.Population).counter()')[0]
for x in counter:
    print('population %s found %s times' % (x.key, x.count))

population ESN found 32 times
population GIH found 30 times
population CHS found 30 times
population PEL found 30 times
population PUR found 24 times
population BEB found 27 times
population MXL found 17 times
population GWD found 39 times
population ACB found 33 times
population PJL found 30 times
population GBR found 33 times
population IBS found 22 times
population YRI found 24 times
population LWK found 28 times
population KHV found 25 times
population FIN found 23 times
population ASW found 19 times
population JPT found 35 times
population CEU found 31 times
population CLM found 25 times
population STU found 22 times
population TSI found 31 times
population CHB found 33 times
population CDX found 26 times
population ITU found 23 times
population MSL found 18 times

###VCF file contains many annotations scores that define the quality of genotypes and variants, which allows genotype and varaint filtering

In [30]:
%scala
displayHTML("https://en.wikipedia.org/wiki/Phred_quality_score")

https://en.wikipedia.org/wiki/Phred_quality_score

###Let's first filter genotypes based on genotype quality (GQ) and read coverage (DP).

In [32]:
filter_condition_gDP_gGQ = 'g.dp >= 5 && g.gq >= 20'
vds_gDP_gGQ = vds.filter_genotypes(filter_condition_gDP_gGQ)

In [33]:
vds_gDP_gGQ.count(genotypes=True)

Out[ 104 ]: {u'nGenotypes': 7782310L, u'nVariants': 10961L, u'nSamples': 710, u'nCalled': 4214198L, u'callRate': 54.15099115815227}

Now the call rate is about 54%, so close to 45% genotypes failed the filter. Filtering out a genotype is equivalent to setting the genotype call to missing.

###Let's next remove variants with low call rate (vCR)

In [36]:
vds_gDP_gGQ_vCR = (vds_gDP_gGQ
    .filter_variants_expr('gs.fraction(g => g.isCalled) >= 0.40')
    .sample_qc())

Check how many variants retained after filtering.

In [38]:
vds_gDP_gGQ_vCR.count(genotypes=True)

Out[ 106 ]: {u'nGenotypes': 6936700L, u'nVariants': 9770L, u'nSamples': 710, u'nCalled': 3939258L, u'callRate': 56.78864589790535}

### Filter samples according to their average read depth (sDP) and genotype quality (sGQ)

In [40]:
print(vds_gDP_gGQ_vCR.sample_schema)

Struct {
 myAnnot: Struct {
 Sample: String,
 Population: String,
 SuperPopulation: String,
 isFemale: Boolean,
 PurpleHair: Boolean,
 CaffeineConsumption: Double
 },
 qc: Struct {
 callRate: Double,
 nCalled: Int,
 nNotCalled: Int,
 nHomRef: Int,
 nHet: Int,
 nHomVar: Int,
 nSNP: Int,
 nInsertion: Int,
 nDeletion: Int,
 nSingleton: Int,
 nTransition: Int,
 nTransversion: Int,
 dpMean: Double,
 dpStDev: Double,
 gqMean: Double,
 gqStDev: Double,
 nNonRef: Int,
 rTiTv: Double,
 rHetHomVar: Double,
 rInsertionDeletion: Double
 }
}

In [41]:
sampleqc_table = vds_gDP_gGQ_vCR.samples_keytable().to_dataframe()

new_column_name_list= list(map(lambda x: x.replace(".", "_"), sampleqc_table.columns))
sampleqc_table = sampleqc_table.toDF(*new_column_name_list)

In [42]:
display(sampleqc_table)

s,sa_myAnnot_Sample,sa_myAnnot_Population,sa_myAnnot_SuperPopulation,sa_myAnnot_isFemale,sa_myAnnot_PurpleHair,sa_myAnnot_CaffeineConsumption,sa_qc_callRate,sa_qc_nCalled,sa_qc_nNotCalled,sa_qc_nHomRef,sa_qc_nHet,sa_qc_nHomVar,sa_qc_nSNP,sa_qc_nInsertion,sa_qc_nDeletion,sa_qc_nSingleton,sa_qc_nTransition,sa_qc_nTransversion,sa_qc_dpMean,sa_qc_dpStDev,sa_qc_gqMean,sa_qc_gqStDev,sa_qc_nNonRef,sa_qc_rTiTv,sa_qc_rHetHomVar,sa_qc_rInsertionDeletion
HG00096,HG00096,GBR,EUR,false,false,77.0,0.2749232343909928,2686,7084,1130,1248,308,1864,0,0,1,1524,340,7.4586746090841425,1.8291623838959317,45.90655249441547,28.46876215640281,1556,4.482352941176471,4.0519480519480515,null
HG00100,HG00100,GBR,EUR,true,false,59.0,0.9616171954964176,9395,375,5081,2779,1535,5849,0,0,2,4762,1087,13.224374667376209,3.712169739189841,55.6111761575305,27.065478772644916,4314,4.380864765409384,1.8104234527687297,null
HG00105,HG00105,GBR,EUR,false,false,77.0,0.5797338792221085,5664,4106,2793,2024,847,3718,0,0,0,2993,725,8.650600282485852,2.2771253327175165,44.382591807909634,27.390137262730732,2871,4.128275862068966,2.3896103896103895,null
HG00114,HG00114,GBR,EUR,false,true,72.0,0.14943705220061412,1460,8310,509,776,175,1126,0,0,0,924,202,6.986301369863015,1.6055823053687388,43.37945205479454,25.086450301723687,951,4.574257425742574,4.434285714285714,null
HG00115,HG00115,GBR,EUR,false,true,71.0,0.6856704196519959,6699,3071,3479,2218,1002,4222,0,0,1,3451,771,9.22779519331246,2.5460202117218307,44.704433497536925,26.70956053033648,3220,4.476005188067445,2.2135728542914173,null
HG00116,HG00116,GBR,EUR,false,false,71.0,0.25220061412487205,2464,7306,1065,1075,324,1723,0,0,0,1403,320,7.394074675324675,1.7025517387131073,40.977678571428584,23.627226861323056,1399,4.384375,3.317901234567901,null
HG00119,HG00119,GBR,EUR,false,true,70.0,0.3195496417604913,3122,6648,1363,1365,394,2153,0,0,0,1743,410,7.527866752082009,1.7771483099316476,42.03939782190902,24.84769850706469,1759,4.251219512195122,3.464467005076142,null
HG00120,HG00120,GBR,EUR,true,true,68.0,0.28607983623336747,2795,6975,1238,1241,316,1873,0,0,0,1517,356,7.446153846153844,1.6802170140360537,41.18246869409654,23.966680366447953,1557,4.26123595505618,3.9272151898734178,null
HG00128,HG00128,GBR,EUR,true,true,67.0,0.6401228249744114,6254,3516,3277,2036,941,3918,0,0,0,3153,765,9.280620402942137,2.5949974996440823,45.32523185161495,27.475594538376384,2977,4.12156862745098,2.1636556854410203,null
HG00131,HG00131,GBR,EUR,false,false,72.0,0.309007164790174,3019,6751,1311,1317,391,2099,0,0,0,1712,387,7.452798940046386,1.6856292863721216,41.58860549850948,24.326264763553834,1708,4.423772609819121,3.368286445012788,null


In [43]:
display(sampleqc_table)

s,sa_myAnnot_Sample,sa_myAnnot_Population,sa_myAnnot_SuperPopulation,sa_myAnnot_isFemale,sa_myAnnot_PurpleHair,sa_myAnnot_CaffeineConsumption,sa_qc_callRate,sa_qc_nCalled,sa_qc_nNotCalled,sa_qc_nHomRef,sa_qc_nHet,sa_qc_nHomVar,sa_qc_nSNP,sa_qc_nInsertion,sa_qc_nDeletion,sa_qc_nSingleton,sa_qc_nTransition,sa_qc_nTransversion,sa_qc_dpMean,sa_qc_dpStDev,sa_qc_gqMean,sa_qc_gqStDev,sa_qc_nNonRef,sa_qc_rTiTv,sa_qc_rHetHomVar,sa_qc_rInsertionDeletion
HG00096,HG00096,GBR,EUR,false,false,77.0,0.2749232343909928,2686,7084,1130,1248,308,1864,0,0,1,1524,340,7.4586746090841425,1.8291623838959317,45.90655249441547,28.46876215640281,1556,4.482352941176471,4.0519480519480515,null
HG00100,HG00100,GBR,EUR,true,false,59.0,0.9616171954964176,9395,375,5081,2779,1535,5849,0,0,2,4762,1087,13.224374667376209,3.712169739189841,55.6111761575305,27.065478772644916,4314,4.380864765409384,1.8104234527687297,null
HG00105,HG00105,GBR,EUR,false,false,77.0,0.5797338792221085,5664,4106,2793,2024,847,3718,0,0,0,2993,725,8.650600282485852,2.2771253327175165,44.382591807909634,27.390137262730732,2871,4.128275862068966,2.3896103896103895,null
HG00114,HG00114,GBR,EUR,false,true,72.0,0.14943705220061412,1460,8310,509,776,175,1126,0,0,0,924,202,6.986301369863015,1.6055823053687388,43.37945205479454,25.086450301723687,951,4.574257425742574,4.434285714285714,null
HG00115,HG00115,GBR,EUR,false,true,71.0,0.6856704196519959,6699,3071,3479,2218,1002,4222,0,0,1,3451,771,9.22779519331246,2.5460202117218307,44.704433497536925,26.70956053033648,3220,4.476005188067445,2.2135728542914173,null
HG00116,HG00116,GBR,EUR,false,false,71.0,0.25220061412487205,2464,7306,1065,1075,324,1723,0,0,0,1403,320,7.394074675324675,1.7025517387131073,40.977678571428584,23.627226861323056,1399,4.384375,3.317901234567901,null
HG00119,HG00119,GBR,EUR,false,true,70.0,0.3195496417604913,3122,6648,1363,1365,394,2153,0,0,0,1743,410,7.527866752082009,1.7771483099316476,42.03939782190902,24.84769850706469,1759,4.251219512195122,3.464467005076142,null
HG00120,HG00120,GBR,EUR,true,true,68.0,0.28607983623336747,2795,6975,1238,1241,316,1873,0,0,0,1517,356,7.446153846153844,1.6802170140360537,41.18246869409654,23.966680366447953,1557,4.26123595505618,3.9272151898734178,null
HG00128,HG00128,GBR,EUR,true,true,67.0,0.6401228249744114,6254,3516,3277,2036,941,3918,0,0,0,3153,765,9.280620402942137,2.5949974996440823,45.32523185161495,27.475594538376384,2977,4.12156862745098,2.1636556854410203,null
HG00131,HG00131,GBR,EUR,false,false,72.0,0.309007164790174,3019,6751,1311,1317,391,2099,0,0,0,1712,387,7.452798940046386,1.6856292863721216,41.58860549850948,24.326264763553834,1708,4.423772609819121,3.368286445012788,null


In [44]:
vds_gDP_gGQ_vCR_sDP_sGQ = (vds_gDP_gGQ_vCR
    .annotate_samples_vds(vds_gDP_gGQ_vCR, code = 'sa.qc = vds.qc' )
    .filter_samples_expr('sa.qc.dpMean >=5 && sa.qc.gqMean >=40'))

As before, we can count the number of samples that remain in the dataset after filtering. (But nothing has been filtered out here)

In [46]:
vds_gDP_gGQ_vCR_sDP_sGQ.count(genotypes=True)

Out[ 108 ]: {u'nGenotypes': 6711990L, u'nVariants': 9770L, u'nSamples': 687, u'nCalled': 3856473L, u'callRate': 57.456477140162605}

### More detailed variants filtering (vFilter) according to their distribution plots

We now have `vds_gDP_gGQ_vCR_sDP_sGT`, a VDS where low-quality genotypes and samples have been removed.

Let's use the [variant_qc](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.variant_qc) method to start exploring variant metrics to perform more detailed variant filtering:

In [48]:
vds_gDP_gGQ_vCR_sDP_sGQ = vds_gDP_gGQ_vCR_sDP_sGQ.split_multi()
vds_gDP_gGQ_vCR_sDP_sGQ = vds_gDP_gGQ_vCR_sDP_sGQ.variant_qc()
print(vds_gDP_gGQ_vCR_sDP_sGQ.variant_schema)

Struct {
 rsid: String,
 qual: Double,
 filters: Set[String],
 pass: Boolean,
 info: Struct {
 AC: Array[Int],
 AF: Array[Double],
 AN: Int,
 BaseQRankSum: Double,
 ClippingRankSum: Double,
 DP: Int,
 DS: Boolean,
 FS: Double,
 HaplotypeScore: Double,
 InbreedingCoeff: Double,
 MLEAC: Array[Int],
 MLEAF: Array[Double],
 MQ: Double,
 MQ0: Int,
 MQRankSum: Double,
 QD: Double,
 ReadPosRankSum: Double,
 set: String
 },
 aIndex: Int,
 wasSplit: Boolean,
 qc: Struct {
 callRate: Double,
 AC: Int,
 AF: Double,
 nCalled: Int,
 nNotCalled: Int,
 nHomRef: Int,
 nHet: Int,
 nHomVar: Int,
 dpMean: Double,
 dpStDev: Double,
 gqMean: Double,
 gqStDev: Double,
 nNonRef: Int,
 rHeterozygosity: Double,
 rHetHomVar: Double,
 rExpectedHetFrequency: Double,
 pHWE: Double
 }
}

In [49]:
variantqc_table = vds_gDP_gGQ_vCR_sDP_sGQ.variants_keytable().to_dataframe()
display(variantqc_table)

v.contig,v.start,v.ref,v.altAlleles,va.rsid,va.qual,va.filters,va.pass,va.info.AC,va.info.AF,va.info.AN,va.info.BaseQRankSum,va.info.ClippingRankSum,va.info.DP,va.info.DS,va.info.FS,va.info.HaplotypeScore,va.info.InbreedingCoeff,va.info.MLEAC,va.info.MLEAF,va.info.MQ,va.info.MQ0,va.info.MQRankSum,va.info.QD,va.info.ReadPosRankSum,va.info.set,va.aIndex,va.wasSplit,va.qc.callRate,va.qc.AC,va.qc.AF,va.qc.nCalled,va.qc.nNotCalled,va.qc.nHomRef,va.qc.nHet,va.qc.nHomVar,va.qc.dpMean,va.qc.dpStDev,va.qc.gqMean,va.qc.gqStDev,va.qc.nNonRef,va.qc.rHeterozygosity,va.qc.rHetHomVar,va.qc.rExpectedHetFrequency,va.qc.pHWE
1,904165,G,"List(List(G, A))",.,52346.37000000001,List(),false,List(518),List(0.103),5020,-3.394,-0.17,17827,null,2.233,null,0.0988,List(514),List(0.102),59.05,0,1.447,15.02,6.286,null,1,false,0.5181950509461426,92,0.12921348314606743,356,331,272,76,8,9.870786516853931,3.2099109948524105,39.808988764044955,23.730877058037567,84,0.21348314606741572,9.5,0.2253512223644495,0.29003883454748414
1,1707740,T,"List(List(T, G))",.,93517.82,List(),false,List(997),List(0.198),5034,-40.42,-0.287,19902,null,3.311,null,0.0387,List(983),List(0.195),58.32,0,9.478,13.59,2.259,null,1,false,0.6259097525473072,186,0.21627906976744185,430,257,264,146,20,10.202325581395353,3.507267429813318,47.57906976744185,27.84244758469988,166,0.3395348837209302,7.3,0.33939951809838376,0.9433988035930663
1,2284195,T,"List(List(T, C))",.,142480.77,List(),false,List(1559),List(0.312),4990,-45.982,0.35,18176,null,2.945,null,0.0925,List(1552),List(0.311),58.57,0,16.136,15.48,-0.682,null,1,false,0.5749636098981077,265,0.33544303797468356,395,292,174,177,44,10.124050632911395,3.8222935387239034,50.76455696202529,28.571877108663653,221,0.4481012658227848,4.0227272727272725,0.44640708475718344,0.9551366128713483
1,2944527,G,"List(List(G, A))",.,124329.15,List(),false,List(1206),List(0.245),4928,0.063,-0.655,17698,null,0.449,null,0.1232,List(1192),List(0.242),58.15,0,12.011,17.54,21.754,null,1,false,0.5342066957787481,191,0.26021798365122617,367,320,199,145,23,10.21798365122616,3.7316531418923233,50.100817438692076,28.092551872719575,168,0.39509536784741145,6.304347826086956,0.3855344205255547,0.6363296264457586
1,3761547,C,"List(List(C, A))",.,1614.69,List(),false,List(30),List(0.005948),5044,-4.468,-8.82,16845,null,2.055,null,-0.0047,List(28),List(0.005551),56.98,0,6.299,7.99,-1.752,null,1,false,0.5443959243085881,10,0.013368983957219251,374,313,364,10,0,8.79411764705883,2.938381024472951,35.890374331550824,15.117941438430414,10,0.026737967914438502,null,0.02641582372269828,0.529558260774204
1,3803755,T,"List(List(T, C))",.,383548.94,List(),false,List(3368),List(0.673),5008,-53.782,-6.841,17687,null,9.582,null,0.0812,List(3376),List(0.674),58.0,0,26.256,24.93,-0.433,null,1,false,0.5662299854439592,522,0.6709511568123393,389,298,41,174,174,9.347043701799482,3.3586588017654098,50.33676092544985,28.10780222298778,348,0.4473007712082262,1.0,0.44211968119423134,0.8638672568716725
1,4121584,A,"List(List(A, G))",.,115117.70000000001,List(),false,List(1489),List(0.3),4962,-26.975,-0.719,16179,null,22.674,null,0.104,List(1465),List(0.295),58.55,0,2.845,14.32,-4.907,null,1,false,0.5502183406113537,227,0.3002645502645503,378,309,188,153,37,9.235449735449736,3.263573597825833,49.96560846560851,27.783105183267022,190,0.40476190476190477,4.135135135135135,0.42076807176144926,0.42779227294020966
1,4170048,C,"List(List(C, T))",.,120311.95,List(),false,List(1323),List(0.266),4980,-12.472,-10.32,16375,null,4.365,null,0.1302,List(1313),List(0.264),58.96,0,7.409,16.72,1.056,null,1,false,0.49199417758369723,176,0.2603550295857988,338,349,176,148,14,9.446745562130182,3.5491233270883495,48.57100591715978,27.46707527990407,162,0.4378698224852071,10.571428571428571,0.3857111549419242,0.01340159575173201
1,4180842,C,"List(List(C, T))",.,138252.12,List(),false,List(1429),List(0.286),4996,54.319,-0.207,15845,null,0.321,null,0.0815,List(1430),List(0.286),59.15,0,-1.479,17.99,-1.476,null,1,false

In [50]:
display(variantqc_table)

v.contig,v.start,v.ref,v.altAlleles,va.rsid,va.qual,va.filters,va.pass,va.info.AC,va.info.AF,va.info.AN,va.info.BaseQRankSum,va.info.ClippingRankSum,va.info.DP,va.info.DS,va.info.FS,va.info.HaplotypeScore,va.info.InbreedingCoeff,va.info.MLEAC,va.info.MLEAF,va.info.MQ,va.info.MQ0,va.info.MQRankSum,va.info.QD,va.info.ReadPosRankSum,va.info.set,va.aIndex,va.wasSplit,va.qc.callRate,va.qc.AC,va.qc.AF,va.qc.nCalled,va.qc.nNotCalled,va.qc.nHomRef,va.qc.nHet,va.qc.nHomVar,va.qc.dpMean,va.qc.dpStDev,va.qc.gqMean,va.qc.gqStDev,va.qc.nNonRef,va.qc.rHeterozygosity,va.qc.rHetHomVar,va.qc.rExpectedHetFrequency,va.qc.pHWE
1,904165,G,"List(List(G, A))",.,52346.37000000001,List(),false,List(518),List(0.103),5020,-3.394,-0.17,17827,null,2.233,null,0.0988,List(514),List(0.102),59.05,0,1.447,15.02,6.286,null,1,false,0.5181950509461426,92,0.12921348314606743,356,331,272,76,8,9.870786516853931,3.2099109948524105,39.808988764044955,23.730877058037567,84,0.21348314606741572,9.5,0.2253512223644495,0.29003883454748414
1,1707740,T,"List(List(T, G))",.,93517.82,List(),false,List(997),List(0.198),5034,-40.42,-0.287,19902,null,3.311,null,0.0387,List(983),List(0.195),58.32,0,9.478,13.59,2.259,null,1,false,0.6259097525473072,186,0.21627906976744185,430,257,264,146,20,10.202325581395353,3.507267429813318,47.57906976744185,27.84244758469988,166,0.3395348837209302,7.3,0.33939951809838376,0.9433988035930663
1,2284195,T,"List(List(T, C))",.,142480.77,List(),false,List(1559),List(0.312),4990,-45.982,0.35,18176,null,2.945,null,0.0925,List(1552),List(0.311),58.57,0,16.136,15.48,-0.682,null,1,false,0.5749636098981077,265,0.33544303797468356,395,292,174,177,44,10.124050632911395,3.8222935387239034,50.76455696202529,28.571877108663653,221,0.4481012658227848,4.0227272727272725,0.44640708475718344,0.9551366128713483
1,3761547,C,"List(List(C, A))",.,1614.69,List(),false,List(30),List(0.005948),5044,-4.468,-8.82,16845,null,2.055,null,-0.0047,List(28),List(0.005551),56.98,0,6.299,7.99,-1.752,null,1,false,0.5443959243085881,10,0.013368983957219251,374,313,364,10,0,8.79411764705883,2.938381024472951,35.890374331550824,15.117941438430414,10,0.026737967914438502,null,0.02641582372269828,0.529558260774204
1,3803755,T,"List(List(T, C))",.,383548.94,List(),false,List(3368),List(0.673),5008,-53.782,-6.841,17687,null,9.582,null,0.0812,List(3376),List(0.674),58.0,0,26.256,24.93,-0.433,null,1,false,0.5662299854439592,522,0.6709511568123393,389,298,41,174,174,9.347043701799482,3.3586588017654098,50.33676092544985,28.10780222298778,348,0.4473007712082262,1.0,0.44211968119423134,0.8638672568716725
1,4121584,A,"List(List(A, G))",.,115117.70000000001,List(),false,List(1489),List(0.3),4962,-26.975,-0.719,16179,null,22.674,null,0.104,List(1465),List(0.295),58.55,0,2.845,14.32,-4.907,null,1,false,0.5502183406113537,227,0.3002645502645503,378,309,188,153,37,9.235449735449736,3.263573597825833,49.96560846560851,27.783105183267022,190,0.40476190476190477,4.135135135135135,0.42076807176144926,0.42779227294020966
1,4170048,C,"List(List(C, T))",.,120311.95,List(),false,List(1323),List(0.266),4980,-12.472,-10.32,16375,null,4.365,null,0.1302,List(1313),List(0.264),58.96,0,7.409,16.72,1.056,null,1,false,0.49199417758369723,176,0.2603550295857988,338,349,176,148,14,9.446745562130182,3.5491233270883495,48.57100591715978,27.46707527990407,162,0.4378698224852071,10.571428571428571,0.3857111549419242,0.01340159575173201
1,4180842,C,"List(List(C, T))",.,138252.12,List(),false,List(1429),List(0.286),4996,54.319,-0.207,15845,null,0.321,null,0.0815,List(1430),List(0.286),59.15,0,-1.479,17.99,-1.476,null,1,false,0.519650655021834,223,0.3123249299719888,357,330,158,175,24,8.700280112044819,2.876997526498,49.128851540616246,28.410066115097898,199,0.49019607843137253,7.291666666666667,0.43015859920405747,0.008024727052244466
1,6279383,G,"List(List(G, C))",.,1268.87,List(),false,List(16),List(0.003197),5004,-4.319,-1.24,15942,null,3.279,null,-0.028,List(15),List(0.002998),58.22,0,0.258,10.75,1.466,null,1,false,0.43

In [51]:
vds_gDP_gGQ_vCR_sDP_sGQ_vFilter = vds_gDP_gGQ_vCR_sDP_sGQ.filter_variants_expr('va.info.MQ >= 55.00 && va.info.QD >= 2.00 && va.info.FS <= 60.000 && va.info.MQRankSum >= -20.000 && va.info.ReadPosRankSum >= -10.000 && va.info.ReadPosRankSum <= 10.000')
print('variants before filtering: %d' % vds_gDP_gGQ_vCR_sDP_sGQ.count_variants())
print('variants after filtering: %d' % vds_gDP_gGQ_vCR_sDP_sGQ_vFilter.count_variants())

variants before filtering: 9770
variants after filtering: 9205

In [52]:
variantqc_table = vds_gDP_gGQ_vCR_sDP_sGQ_vFilter.variants_keytable().to_dataframe()
display(variantqc_table)

v.contig,v.start,v.ref,v.altAlleles,va.rsid,va.qual,va.filters,va.pass,va.info.AC,va.info.AF,va.info.AN,va.info.BaseQRankSum,va.info.ClippingRankSum,va.info.DP,va.info.DS,va.info.FS,va.info.HaplotypeScore,va.info.InbreedingCoeff,va.info.MLEAC,va.info.MLEAF,va.info.MQ,va.info.MQ0,va.info.MQRankSum,va.info.QD,va.info.ReadPosRankSum,va.info.set,va.aIndex,va.wasSplit,va.qc.callRate,va.qc.AC,va.qc.AF,va.qc.nCalled,va.qc.nNotCalled,va.qc.nHomRef,va.qc.nHet,va.qc.nHomVar,va.qc.dpMean,va.qc.dpStDev,va.qc.gqMean,va.qc.gqStDev,va.qc.nNonRef,va.qc.rHeterozygosity,va.qc.rHetHomVar,va.qc.rExpectedHetFrequency,va.qc.pHWE
1,904165,G,"List(List(G, A))",.,52346.37000000001,List(),false,List(518),List(0.103),5020,-3.394,-0.17,17827,null,2.233,null,0.0988,List(514),List(0.102),59.05,0,1.447,15.02,6.286,null,1,false,0.5181950509461426,92,0.12921348314606743,356,331,272,76,8,9.870786516853931,3.2099109948524105,39.808988764044955,23.730877058037567,84,0.21348314606741572,9.5,0.2253512223644495,0.29003883454748414
1,1707740,T,"List(List(T, G))",.,93517.82,List(),false,List(997),List(0.198),5034,-40.42,-0.287,19902,null,3.311,null,0.0387,List(983),List(0.195),58.32,0,9.478,13.59,2.259,null,1,false,0.6259097525473072,186,0.21627906976744185,430,257,264,146,20,10.202325581395353,3.507267429813318,47.57906976744185,27.84244758469988,166,0.3395348837209302,7.3,0.33939951809838376,0.9433988035930663
1,2284195,T,"List(List(T, C))",.,142480.77,List(),false,List(1559),List(0.312),4990,-45.982,0.35,18176,null,2.945,null,0.0925,List(1552),List(0.311),58.57,0,16.136,15.48,-0.682,null,1,false,0.5749636098981077,265,0.33544303797468356,395,292,174,177,44,10.124050632911395,3.8222935387239034,50.76455696202529,28.571877108663653,221,0.4481012658227848,4.0227272727272725,0.44640708475718344,0.9551366128713483
1,3761547,C,"List(List(C, A))",.,1614.69,List(),false,List(30),List(0.005948),5044,-4.468,-8.82,16845,null,2.055,null,-0.0047,List(28),List(0.005551),56.98,0,6.299,7.99,-1.752,null,1,false,0.5443959243085881,10,0.013368983957219251,374,313,364,10,0,8.79411764705883,2.938381024472951,35.890374331550824,15.117941438430414,10,0.026737967914438502,null,0.02641582372269828,0.529558260774204
1,3803755,T,"List(List(T, C))",.,383548.94,List(),false,List(3368),List(0.673),5008,-53.782,-6.841,17687,null,9.582,null,0.0812,List(3376),List(0.674),58.0,0,26.256,24.93,-0.433,null,1,false,0.5662299854439592,522,0.6709511568123393,389,298,41,174,174,9.347043701799482,3.3586588017654098,50.33676092544985,28.10780222298778,348,0.4473007712082262,1.0,0.44211968119423134,0.8638672568716725
1,4121584,A,"List(List(A, G))",.,115117.70000000001,List(),false,List(1489),List(0.3),4962,-26.975,-0.719,16179,null,22.674,null,0.104,List(1465),List(0.295),58.55,0,2.845,14.32,-4.907,null,1,false,0.5502183406113537,227,0.3002645502645503,378,309,188,153,37,9.235449735449736,3.263573597825833,49.96560846560851,27.783105183267022,190,0.40476190476190477,4.135135135135135,0.42076807176144926,0.42779227294020966
1,4170048,C,"List(List(C, T))",.,120311.95,List(),false,List(1323),List(0.266),4980,-12.472,-10.32,16375,null,4.365,null,0.1302,List(1313),List(0.264),58.96,0,7.409,16.72,1.056,null,1,false,0.49199417758369723,176,0.2603550295857988,338,349,176,148,14,9.446745562130182,3.5491233270883495,48.57100591715978,27.46707527990407,162,0.4378698224852071,10.571428571428571,0.3857111549419242,0.01340159575173201
1,4180842,C,"List(List(C, T))",.,138252.12,List(),false,List(1429),List(0.286),4996,54.319,-0.207,15845,null,0.321,null,0.0815,List(1430),List(0.286),59.15,0,-1.479,17.99,-1.476,null,1,false,0.519650655021834,223,0.3123249299719888,357,330,158,175,24,8.700280112044819,2.876997526498,49.128851540616246,28.410066115097898,199,0.49019607843137253,7.291666666666667,0.43015859920405747,0.008024727052244466
1,6279383,G,"List(List(G, C))",.,1268.87,List(),false,List(16),List(0.003197),5004,-4.319,-1.24,15942,null,3.279,null,-0.028,List(15),List(0.002998),58.22,0,0.258,10.75,1.466,null,1,false,0.43

### Finally, we perform PCA on the clean variant dataset

We will use principal component analysis (PCA) to check if there is any genetic structure.

In [54]:
vds_pca = vds_gDP_gGQ_vCR_sDP_sGQ_vFilter.pca(scores='sa.pca')

In [55]:
pca_table = vds_pca.samples_keytable().to_dataframe()
display(pca_table)

s,sa.myAnnot.Sample,sa.myAnnot.Population,sa.myAnnot.SuperPopulation,sa.myAnnot.isFemale,sa.myAnnot.PurpleHair,sa.myAnnot.CaffeineConsumption,sa.qc.callRate,sa.qc.nCalled,sa.qc.nNotCalled,sa.qc.nHomRef,sa.qc.nHet,sa.qc.nHomVar,sa.qc.nSNP,sa.qc.nInsertion,sa.qc.nDeletion,sa.qc.nSingleton,sa.qc.nTransition,sa.qc.nTransversion,sa.qc.dpMean,sa.qc.dpStDev,sa.qc.gqMean,sa.qc.gqStDev,sa.qc.nNonRef,sa.qc.rTiTv,sa.qc.rHetHomVar,sa.qc.rInsertionDeletion,sa.pca.PC1,sa.pca.PC2,sa.pca.PC3,sa.pca.PC4,sa.pca.PC5,sa.pca.PC6,sa.pca.PC7,sa.pca.PC8,sa.pca.PC9,sa.pca.PC10
HG00096,HG00096,GBR,EUR,false,false,77.0,0.2749232343909928,2686,7084,1130,1248,308,1864,0,0,1,1524,340,7.4586746090841425,1.8291623838959317,45.90655249441547,28.46876215640281,1556,4.482352941176471,4.0519480519480515,null,0.033985434118407626,-0.06692902968414734,-3.918654050448722E-5,-0.028172696349145106,-0.01183172602858436,0.017519971166972154,-0.0010459701317010435,0.029602169085176582,-0.0019836075783904996,-0.0017771707280481587
HG00100,HG00100,GBR,EUR,true,false,59.0,0.9616171954964176,9395,375,5081,2779,1535,5849,0,0,2,4762,1087,13.224374667376209,3.712169739189841,55.6111761575305,27.065478772644916,4314,4.380864765409384,1.8104234527687297,null,0.1458948130976565,-0.26401395159821806,0.01580460830513945,-0.12315365761718505,0.02989055969259797,-0.01326122600128924,-0.08366525294650233,-0.13218390575433178,-0.10512962256059674,0.0792733515145119
HG00105,HG00105,GBR,EUR,false,false,77.0,0.5797338792221085,5664,4106,2793,2024,847,3718,0,0,0,2993,725,8.650600282485852,2.2771253327175165,44.382591807909634,27.390137262730732,2871,4.128275862068966,2.3896103896103895,null,0.08966238085580182,-0.13298866140760177,0.018712959720761157,-0.06123033170233247,-0.019221257461995257,0.009909119864810757,0.015166437747194776,-0.006147745699509365,-0.005360653179038534,0.006203062703954431
HG00114,HG00114,GBR,EUR,false,true,72.0,0.14943705220061412,1460,8310,509,776,175,1126,0,0,0,924,202,6.986301369863015,1.6055823053687388,43.37945205479454,25.086450301723687,951,4.574257425742574,4.434285714285714,null,0.016227247555033783,-0.03519444500916458,0.0048659798935873225,-0.017052375149366823,-0.007862523467512095,0.006892077122897193,-0.003898509985393051,0.005546794668680825,0.0012987331760997403,-0.0019128138048738852
HG00115,HG00115,GBR,EUR,false,true,71.0,0.6856704196519959,6699,3071,3479,2218,1002,4222,0,0,1,3451,771,9.22779519331246,2.5460202117218307,44.704433497536925,26.70956053033648,3220,4.476005188067445,2.2135728542914173,null,0.08873362552564232,-0.18644763918990007,-1.150393449029586E-5,-0.04084453646249626,-0.0026755734912689514,0.002720826430351337,-0.012765532645237573,0.050209525113822887,0.013885103961345097,-0.011412734815101258
HG00116,HG00116,GBR,EUR,false,false,71.0,0.25220061412487205,2464,7306,1065,1075,324,1723,0,0,0,1403,320,7.394074675324675,1.7025517387131073,40.977678571428584,23.627226861323056,1399,4.384375,3.317901234567901,null,0.030360474135906007,-0.06001644378018352,0.0060656758150249976,-0.017589138745045772,-0.0018471100897441454,0.007682792286772998,-0.0010094758696741896,0.021272338673214698,0.004779394964683034,-0.0021225561961096956
HG00119,HG00119,GBR,EUR,false,true,70.0,0.3195496417604913,3122,6648,1363,1365,394,2153,0,0,0,1743,410,7.527866752082009,1.7771483099316476,42.03939782190902,24.84769850706469,1759,4.251219512195122,3.464467005076142,null,0.044360694884479475,-0.07150037612853768,8.518414179108384E-4,-0.025523648809213834,-0.0027943270683119257,0.013170726352807721,-0.00541858365128325,0.02080287369244803,-0.002365140755455032,-0.006058727913797278
HG00120,HG00120,GBR,EUR,true,true,68.0,0.28607983623336747,2795,6975,1238,1241,316,1873,0,0,0,1517,356,7.446153846153844,1.6802170140360537,41.18246869409654,23.966680366447953,1557,4.26123595505618,3.9272151898734178,null,0.03488941547990683,-0.0762717583295123,0.003671037884688223,-0.03069365338417546,-0.005691016826978085,0.009657848628715103,-0.0011829546882877395,0.0135604981440

## Summary:

Data filtering:
 - Filter genotypes
 - Filter samples
 - Filter variants

PCA 
 - Genetic structure of super-poupulations